In [1]:
#include "../common.hpp"

namespace bcc { }
using namespace bcc;

# Efficiency & Expressiveness

## Recap

### Definition of _Type_ and _Regular_

> A _type_ is a pattern for storing and modifying objects.

<div></div>

> There is a set of procedures whose inclusion in the computational basis of a type lets us place objects in data structures and use algorithms to _copy objects_ from one data structure to another. We call types having such a basis _regular_, since their use guarantees regularity of behavior and, therefore, interoperability.

### Semantics and Complexity

- Semantics of an operation are defined with axioms derived from the definition. i.e. we can define the semantics of equality as:

\begin{align}
(\forall a) a & = a. && \text{(Reflexivity)} \\
(\forall a, b) a & = b \implies b = a. && \text{(Symmetry)} \\
(\forall a, b, c) a & = b \wedge b = c \implies a = c. && \text {(Transitivity)} \\
\end{align}

- This covers any equivalence relation
- Two objects are _equal_ iff their two values represent the same entity

- The expected complexity of an operation is an important attribute of the operation
- i.e. The only thing that separates the concept of `ForwardIterator` and `RandomAccessIterator` is the complexity of advancing `n` steps

### Computational Basis and Computationally Complete
> The _computational basis_ for a type is a finite set of procedures that enable the construction of any other procedure on the type

- A type which does not implement a _computational basis_ is _incomplete_

### Equationally Complete
- A type for which equality can be implemented as a non-friend (non-member) function is said to be _equationally complete_
- A type which is both equationally and computationally complete can be copied without the use of the copy-constructor or assignment operator
    - Equationally complete implies all the parts are readable
    - Computationally complete implies all the values are obtainable
    

### Whole/Part Relationship
- An object is a _whole_, composed of its _parts_
- A part is _local_ if it is stored directly in the object
- A part is _remote_ if it is stored elsewhere (such as on the heap)

### Safety
- Any operation which maintains the correspondence between an object and an entity it represents is _safe_
- An operation which loses the correspondence is _unsafe_

### Canonical Type with and without Remote Parts

In [2]:
namespace bcc {

class simple_type {
    int _members = 0;

public:
    simple_type() noexcept = default;                         // default-ctor

    simple_type(const simple_type&) = default;                // copy-ctor
    simple_type& operator=(const simple_type&) = default;     // copy-assign

    simple_type(simple_type&&) noexcept = default;            // move-ctor
    simple_type& operator=(simple_type&&) noexcept = default; // move_assign

    friend bool operator==(const simple_type& a, const simple_type& b) {
        return tie(a._members /*, ...*/) == tie(b._members /*, ...*/);
    }
    friend bool operator!=(const simple_type& a, const simple_type& b) {
        return !(a == b);
    }
};

} // namespace bcc

In [3]:
namespace bcc {

class pimpl_type {
    class implementation;
    struct deleter {
        void operator()(implementation*) const;
    };
    unique_ptr<implementation, deleter> _remote;

public:
    pimpl_type() noexcept = default;                        // default-ctor
    pimpl_type(const pimpl_type&);                          // copy-ctor
    pimpl_type& operator=(const pimpl_type& a) {            // copy-assign
        return *this = pimpl_type(a);
    }
    pimpl_type(pimpl_type&&) noexcept = default;            // move-ctor
    pimpl_type& operator=(pimpl_type&&) noexcept = default; // move_assign
    friend bool operator==(const pimpl_type&, const pimpl_type&);
    friend bool operator!=(const pimpl_type& a, const pimpl_type& b) {
        return !(a == b);
    }
};

} // namespace bcc

In [4]:
// cpp file
namespace bcc {

struct pimpl_type::implementation {
    // a simple type...
    int _members = 0;

    friend bool operator==(const implementation& a, const implementation& b) {
        return tie(a._members /*, ...*/) == tie(b._members /*, ...*/);
    }
};

void pimpl_type::deleter::operator()(implementation* a) const { delete a; }

pimpl_type::pimpl_type(const pimpl_type& a)
    : _remote(new implementation(*a._remote)) {}

bool operator==(const pimpl_type& a, const pimpl_type& b) {
    return *a._remote == *b._remote;
}

} // namespace bcc

- In both cases the default-dtor is used (not specified)
- We will be covering polymorphic types and containers later in the course

## Prior Homework

**Exercise** Look at the regular operations (copy, assignment, equality, default construction) for a type in the standard library, or a commonly used type within your project. Is the implementation correct? Complete? Efficient?

## Prior Homework

**Exercise:** Look at the regular operations (copy, assignment, equality, default construction) for ZString (or a commonly used type within your project). Is the implementation correct? Complete? Efficient?

`ZString` operations:
- default-ctor: Should be declared `noexcept` but will not throw
```cpp
    ZString();
```
- copy-ctor: Logical copy by incrementing reference count to immutable string, should be declared `noexcept`.
```cpp
    ZString(const ZString &x);
```
- copy-assign: Handles self assignment, requires locking (spin-lock). Complex logic. Benchmark against a copy/move implementation? Returns void?
```cpp
    void operator=(const ZString &x);
```
- move-ctor: Should be declared `noexcept` but will not throw, expensive operation to atomic increment a reference count on `TheOneTrueEmptyZByteRun`, guarantees moved from `ZString` is empty string.
```cpp
    ZString(ZString&& x);
```

- move-assign: Implemented as swap(). Does not guarantee moved from `ZString` is empty.
```cpp
    ZString& operator=(ZString&& x) noexcept;
```
- equality: Representational (not value) equality. Should be declared as non-member function.
```cpp
    bool operator == (const ZString &x) const;
```

Observation: `fDefaultRun` is hardly used except for test cases and to propagate `fCharacterRun`. Is it needed?

Discussion: How can we incrementally improve ZString?

## What Should Be Part of The Public Interface On A Type?

- In general we want the minimum number of public calls with private access to provide a type which is:
    - Computationally Complete
    - Equationally Complete
    
- Other operations should be implemented in terms of those

Example:

In [5]:
namespace bcc {

class number {
    unsigned int _data = 0;

public:
    // default standard operations
    number& operator++() {
        ++_data;
        return *this;
    }
    friend unsigned int operator-(const number& a, const number& b) {
        return a._data - b._data;
    }
};

} // namespace bcc

- `number` is computationally and equationally complete

In [6]:
namespace bcc {

bool operator==(const number& a, const number& b) {
    return (a - b) == 0;
}

} // namespace bcc

- Being correct and complete is not enough:

In [7]:
{
// construct the value 3
number a; ++a; ++a; ++a;

// print it
cout << (a - number()) << endl;
}

3


## Efficient Basis

- An operation is _efficient_ if there is no way to implement it to use fewer resources:
    - time
    - space
    - energy
    
- Unless otherwise specified, we will use efficiency to mean _time efficiency_
    - But in practice, where not all three can be achieved the trade-offs should be considered

- A type has an _efficient basis_ if any additional operations can be implemented efficiently in terms of the basis operations

- Making all data members public ensures an efficient basis, but may be unsafe
- In fact, we can prove that some operations cannot be implemented both efficiently and safely
- The canonical example is in-situ sort, although it is true of any in-situ permutation
    - This is why functional languages do not allow direct in-situ permutations

- In C++, explicit `move` is both unsafe and inefficient
    - It is less safe than copy
    - But more efficient than copy
    
- Strive to make operations safe _and_ efficient
- Only sacrifice safety for efficiency with good (measurable) reason

## Expressive Basis

> A basis is _expressive_ iff it allows compact and convenient definitions of procedures on the type.

For example:

In [8]:
{
// construct the value 3
number a; ++a; ++a; ++a;
}

is not as expressive as:

In [9]:
{
int a = 3;
}

- Especially for common operators you should provide operations in meaningful groups:
- If your provide `operator==()` (and you should), also provide `!=`
- If you provide `operator<()`, _natural total order_, you should provide all comparison operators
- Negation and addition implies subtraction
- etc.

## (Revisited) What Should Be Part of The Public Interface On A Type?

- In general we want the **minimum** number of public calls with private access to provide a type which is:
    - Computationally Complete
    - Equationally Complete
    - Efficient
    - Safe
    - Operations required to be part of the class interface by the language (i.e., you cannot implement a stand alone assignment operator)
    
- Other operations, including operations that are part of the expressive basis, should be implemented in terms of those operations
- This still leaves a fair amount up to the designer to choose how to balance safety and efficiency and what _expressive_ means in the context of the type

**Exercise:** Look at the API and implementation for ZString (or a commonly used class in your own project). What does a ZString represent? What would be a good set of basis operations? What operations would be better implemented externally? Are there operations that should be removed?